# Bag of Words Meets Bags of Popcorn

This is my second Kaggle competition. This is tutorial competition to learn about Word2vec neural network implementation but I realized project depending on tips for applying a simple Bag of Words model, mentioned in the tutorial. 

Link: https://www.kaggle.com/c/word2vec-nlp-tutorial

Problem description: need to create model for sentiment analysis of movie reviews. The model must distinguish negative and positive movie reviews and mark them as 0 and 1 accordingly. (NLP problem)

## 1 - Packages

In [ ]:
import pandas as pd 
import numpy as np
import re
import seaborn as sns
from bs4 import BeautifulSoup

from sklearn import model_selection, ensemble, metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

import nltk
nltk.download('stopwords', 'wordnet', 'punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## 2 - Overview of the Dataset

Loading data:

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test_data = pd.read_csv("../input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
submit_data = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3)

Shape of datasets:

In [ ]:
print('Shape of train data: ', train_data.shape)
print('Shape of test data: ', test_data.shape)
print('Shape of submit data: ', submit_data.shape)

First 5 rows of train data dataset:

In [ ]:
train_data.head()

Check column names:

In [ ]:
print('Column names:', list(train_data.columns))

Let's see review samples.

In [ ]:
train_data.review[0]

Text has HTML tags, punctuation, stop words(such as “the”, “a”, “an”, “in”) and numbers. So, we need to clean our data and then tokenize it.

## 3 - Data Cleaning and Text Preprocessing

Next functions are created for data cleaning. Preprocessing function prepares a review for following splitting into tokens. Tokenizing function works with every word in review to get the most accurate tokens for analyzing.

In [ ]:
def my_tokenizer(sample):
    # Split into words
    words = nltk.word_tokenize(sample)
#     print('3',words)
    
    # Leave alphabetical tokens
    tokens = [word for word in words if word.isalnum()]
    tokens = [word for word in tokens if not word.isdigit()]
#     print('4',tokens)
    
    # Remove stopwords
    meaningful_words = [w for w in tokens if not w in stops]
#     print('5',meaningful_words)
    
    # Lemmatization 
    word_list = [lemmatizer.lemmatize(w) for w in meaningful_words]
#     print('6',word_list)
    
    return word_list

def my_preprocessor(sample):
    # Remove HTML tags
    no_tags_text = BeautifulSoup(sample).get_text()  
#     print('1',no_tags_text)
    
    # To lowercase
    review_text = no_tags_text.lower()
#     print('2',review_text)
    
    return review_text

Then we will instantiate lemmatizer and load useful set of stopwords. Now we are ready to instantiate `vectorizer` to perform whole text preprocessing and create bag-of-words features.

In [ ]:
stops = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
vectorizer = CountVectorizer(analyzer = "word", tokenizer = my_tokenizer, preprocessor = my_preprocessor, \
                             stop_words = None, max_features = 5000) 

Let's check work of `vectorizer` on the first sample in out train dataset.

In [ ]:
sample = train_data.review[0]
train_data_features = vectorizer.fit_transform([sample])
train_data_features = train_data_features.toarray()

train_data_features

So, at least it works. I don't really know how to check if it's valid. Now fit our model and transform all train dataset.

In [ ]:
train_data_features = vectorizer.fit_transform(train_data.review)
train_data_features = train_data_features.toarray()

In [ ]:
# test_data_features = vectorizer.transform(test_data.review)
# test_data_features = test_data_features.toarray()

In [ ]:
submit_data_features = vectorizer.transform(submit_data.review)
submit_data_features = submit_data_features.toarray()

The training data array now looks like:

In [ ]:
train_data_features.shape

Now that the Bag of Words model is trained, let's look at the vocabulary:

In [ ]:
vocab = vectorizer.get_feature_names()

## 4 - Model

Below we will consider different models and choose that one, which gives the best metric rate. By the terms, submissions are judged on area under the ROC curve. 

Let's configure:
1. SVM
2. Naive Bayes
3. RandomForestClassifier
4. KNN
5. Logistic Regression

In [ ]:
roc_auc_scorer = metrics.make_scorer(metrics.roc_auc_score)
X_train = train_data_features[:500]
Y_train = train_data.sentiment[:500]

In [ ]:
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train_data.sentiment )

In [ ]:
result = forest.predict(submit_data_features)

### SVM

In [ ]:
params = {'kernel':['linear', 'rbf'], 'C':[0.1, 1, 5, 10]}
svc = SVC(probability = True, random_state = 0)
clf = GridSearchCV(svc, param_grid = params, scoring = roc_auc_scorer, cv = 5, n_jobs = -1)
clf.fit(X_train, Y_train)
print('Best score: {}'.format(clf.best_score_))
print('Best parameters: {}'.format(clf.best_params_))

In [ ]:
svc_best = SVC(C = clf.best_params_['C'], kernel = clf.best_params_['kernel'], probability = True, random_state = 0)

### RandomForestClassifier

In [ ]:
params = {'n_estimators':[10, 50, 100, 150], 'criterion':['gini', 'entropy'], 'max_depth':[None, 5, 10, 50]}
rf = RandomForestClassifier(random_state = 0)
clf = GridSearchCV(rf, param_grid = params, scoring = roc_auc_scorer, cv = 5, n_jobs = -1)
clf.fit(X_train, Y_train)
print('Best score: {}'.format(clf.best_score_))
print('Best parameters: {}'.format(clf.best_params_))

In [ ]:
rf_best = RandomForestClassifier(n_estimators = clf.best_params_['n_estimators'], criterion = clf.best_params_['criterion'], \
                                 max_depth = clf.best_params_['max_depth'], random_state = 0)

### LogisticRegression

In [ ]:
params = {'penalty':['l1', 'l2'], 'C':[1, 2, 3, 5, 10]}
lr = LogisticRegression(random_state = 0)
clf = GridSearchCV(lr, param_grid = params, scoring = roc_auc_scorer, cv = 5, n_jobs = -1)
clf.fit(X_train, Y_train)
print('Best score: {}'.format(clf.best_score_))
print('Best parameters: {}'.format(clf.best_params_))

In [ ]:
lr_best = LogisticRegression(penalty = clf.best_params_['penalty'], C = clf.best_params_['C'], random_state = 0)
# lr_best = LogisticRegression(penalty = 'l2', C = 10, random_state = 0)

### Naive Bayes

In [ ]:
params = {"var_smoothing" : [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]}
nb = GaussianNB()
clf = GridSearchCV(nb, param_grid = params, scoring = roc_auc_scorer, cv = 5, n_jobs = -1)
clf.fit(X_train, Y_train)
print('Best score: {}'.format(clf.best_score_))
print('Best parameters: {}'.format(clf.best_params_))

In [ ]:
nb_best = GaussianNB(var_smoothing = clf.best_params_['var_smoothing'])

### KNN

In [ ]:
params = {'n_neighbors':[3, 5, 10, 20], 'p':[1, 2, 5], 'weights':['uniform', 'distance']}
knc = KNeighborsClassifier()
clf = GridSearchCV(knc, param_grid = params, scoring = roc_auc_scorer, cv = 5, n_jobs = -1)
clf.fit(X_train, Y_train)
print('Best score: {}'.format(clf.best_score_))
print('Best parameters: {}'.format(clf.best_params_))

In [ ]:
knc_best = KNeighborsClassifier(n_neighbors = clf.best_params_['n_neighbors'], p=clf.best_params_['p'],\
                               weights = clf.best_params_['weights'])

In [ ]:
# voting_clf = VotingClassifier(estimators=[('svc', svc_best), ('rf', rf_best), ('lr', lr_best), ('nb', nb_best),\
#                                           ('knc', knc_best)], voting='hard')
# voting_clf.fit(train_data_features, train_data.sentiment)


In [ ]:
lr_best.fit(train_data_features, train_data.sentiment)
y_pred = lr_best.predict(submit_data_features)

Downloading results to file. 

In [ ]:
submission = pd.read_csv("../input/word2vec-nlp-tutorial/sampleSubmission.csv", header=0, delimiter=",", quoting=3)
col = submission.columns[1]
submission[col] = y_pred
submission.to_csv('submission.csv', index=False)

Somewhy this file saved symbols """ instead of ". And I rewrited it in this way:

In [ ]:
f = open("submission.csv", "r")
f.readline()
s = open("valid_submission.csv","w+")
s.write('\"id\",\"sentiment\"\n')
for x in f:
    x = x.split(',')
    x[0] = x[0][2:-2]
    s.write(','.join(x))